In [12]:
import task_monitor_util
import datetime as dt
import simplenote as sn
from io import StringIO
import pandas as pd
import my_util
import matplotlib.pyplot as plt
import ast
from IPython.core.display import display, HTML
reference_date=dt.datetime.now()
reference_date

datetime.datetime(2017, 12, 25, 17, 28, 37, 440102)

In [10]:
task_monitor_util.calculate_n_display_task_n_goal_metrics(reference_date)

Name,Description,Remaining,Points
pricing,Pricing analytics (understand ppts),0.0,1.0
e1d,Eliminate Ek Din (TM),-11.8,1.0
mastery,identify and prioritize mastery opportunity resulting,1.8,1.0
parenting,Identify and act on improving parenting,0.5,1.0
weekend,Identify and act on improving weekends,5.0,0.0
python modeling,End to end model building (analysis n viz n modeling python),10.0,0.0
deep learning,Build deep learning model (normal + timeseries),6.5,1.0


In [7]:
from togglwrapper import Toggl
import pandas as pd
import datetime as dt
from IPython.core.display import display, HTML, display_javascript, Javascript
TOGGL_API_TOKEN='188b4b533d58101a5fcc800f9f00569d'
TOGGL_WORKSPACE_ID=57456
NUMBER_OF_TOGGL_DATA_TO_PULL=30
toggl = Toggl(TOGGL_API_TOKEN)

In [8]:
htm="""
<!DOCTYPE html>
<html lang="en-US">
<script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>

<script type="text/javascript">
// Load google charts
google.charts.load('current', {'packages':['corechart']});
google.charts.setOnLoadCallback(drawChart);

// Draw the chart and set the chart values
function drawChart() {
  var data = google.visualization.arrayToDataTable([
  ['Task', 'Hours per Day'],
  ['Work', 8],
  ['Eat', 2],
  ['TV', 4],
  ['Gym', 2],
  ['Sleep', 8]
]);

  // Optional; add a title and set the width and height of the chart
  var options = {'title':'My Average Day', 'width':550, 'height':400};

  // Display the chart inside the <div> element with id="piechart"
  var chart = new google.visualization.PieChart(document.getElementById('piechart'));
  chart.draw(data, options);
   var chart2 = new google.visualization.PieChart(document.getElementById('piechart2'));
  chart2.draw(data, options);
}
</script>

<body>

<h1>My Web Page</h1>
<table width='90%'><tr><td><div id="piechart"></div></td><td><div id="piechart2"></div></td></tr></table>

</body>
</html>
"""
display(HTML(htm))

,


In [52]:
# Pull Toggl Data
reference_date=dt.datetime.strptime("2017-05-01","%Y-%m-%d")
#reference_date=dt.datetime.now()
start_date=str(reference_date.date()-dt.timedelta(days=NUMBER_OF_TOGGL_DATA_TO_PULL))+"T00:00:00+00:00"
end_date=str(reference_date.date())+"T00:00:00+00:00"

# Pull projects
projects=pd.DataFrame(data=toggl.Workspaces.get_projects(TOGGL_WORKSPACE_ID))
projects=projects.loc[:,['id','name']]

# Pull time entries
time_entries=pd.DataFrame(data=toggl.TimeEntries.get(start_date=start_date,end_date=end_date))
time_entries.stop=pd.to_datetime(time_entries.stop)
time_entries.start=pd.to_datetime(time_entries.start)
time_entries=time_entries.loc[:,['start','description','duration','pid']]
tasks=pd.merge(time_entries,projects,how='left',left_on='pid',right_on='id')
tasks=tasks.loc[:,['start','name','description','duration']]

tasks['Date']=tasks.start.apply(lambda x:dt.datetime(x.year, x.month, x.day))
tasks['EndTime']=tasks.start.apply(lambda x:dt.datetime.strftime(x,"%H:%M"))
tasks['Point']=1
tasks.rename({'name':'Category','description':'Desc','duration':'Duration'},inplace=True,axis=1)
tasks.Category.fillna(value="",inplace=True)
tasks.Category = tasks.Category.apply(lambda x: x.strip().lower())  # remove whitestrips
tasks['DateStr'] = tasks.Date.apply(lambda x: dt.datetime.strftime(x, format="%b-%d"))
tasks['Duration'] = pd.to_numeric(tasks.Duration, errors="coerce")/60  # coerce to numeric if any blank durations
tasks['Duration']=tasks['Duration'].apply(lambda x:round(x))
tasks.drop(labels=["start"],axis=1,inplace=True)
tasks=tasks.loc[:,['Date', 'EndTime', 'Category', 'Desc', 'Duration', 'Point', 'DateStr']]

tasks

,Date,EndTime,Category,Desc,Duration,Point,DateStr
0,2017-04-09,05:00,du,FT,20,1,Apr-09
1,2017-04-09,06:49,du,FT,126,1,Apr-09
2,2017-04-10,04:55,du,FT,224,1,Apr-10
3,2017-04-13,04:49,,tafaqqur,15,1,Apr-13
4,2017-04-13,05:05,du,meeting,55,1,Apr-13
5,2017-04-13,06:00,du,misc,60,1,Apr-13
6,2017-04-17,06:15,du,meeting,90,1,Apr-17
7,2017-04-17,10:30,du,FT,69,1,Apr-17
8,2017-04-18,05:00,du,misc,119,1,Apr-18
9,2017-04-18,07:00,du,meeting,60,1,Apr-18


In [ ]:
pd.rou

In [37]:
tasks

,start,Category,description,duration,Date,EndTime
0,2017-04-09 05:00:13,Du,FT,1202,2017-04-09,05:00
1,2017-04-09 06:49:41,Du,FT,7530,2017-04-09,06:49
2,2017-04-10 04:55:32,Du,FT,13457,2017-04-10,04:55
3,2017-04-13 04:49:48,NaN,tafaqqur,927,2017-04-13,04:49
4,2017-04-13 05:05:03,Du,meeting,3302,2017-04-13,05:05
5,2017-04-13 06:00:31,Du,misc,3601,2017-04-13,06:00
6,2017-04-17 06:15:43,Du,meeting,5416,2017-04-17,06:15
7,2017-04-17 10:30:57,Du,FT,4157,2017-04-17,10:30
8,2017-04-18 05:00:47,Du,misc,7155,2017-04-18,05:00
9,2017-04-18 07:00:24,Du,meeting,3601,2017-04-18,07:00


In [51]:
tasks_all.columns

[Index(['Date', 'EndTime', 'Category', 'Desc', 'Duration', 'Point', 'DateStr'], dtype='object')]

In [25]:
tasks_all

,Date,EndTime,Category,Desc,Duration,Point,DateStr
0,2017-12-21,9:20,quran,rev,60.0,1,Dec-21
1,2017-12-20,9:20,quran,rev,60.0,1,Dec-20
2,2017-12-20,7:27,ilm,tawil ud daim 3-9,20.0,1,Dec-20
3,2017-12-19,10:00,quran,rev,60.0,1,Dec-19
4,2017-12-18,9:30,quran,rev,50.0,1,Dec-18
5,2017-12-14,8:30,quran,rev,30.0,1,Dec-14
6,2017-12-14,3:30,e1d,habit,60.0,1,Dec-14
7,2017-12-13,8:25,quran,rev,60.0,1,Dec-13
8,2017-12-13,10:20,collab,mail,15.0,1,Dec-13
9,2017-12-13,12:10,e1d,habit,60.0,1,Dec-13


In [24]:
tasks_all

Date        datetime64[ns]
EndTime             object
Category            object
Desc                object
Duration           float64
Point                int64
DateStr             object
dtype: object

In [23]:
simplenote = sn.Simplenote('mohdjamal8453@gmail.com', 'simple123')
taskNoteKey = "41d06e8ced6c42389127e0d727974230"
ilmNoteKey = "5f7cb643ec884642b250954ff3996f8c"
ilmRevLogKey = "9ad07e0dc9e84909b17795e101683c68"
monthlyGoalNoteKey = "ba7047bd84204ab49e9e271a5b164675"
def get_tasks(reference_date):
    dat = simplenote.get_note(taskNoteKey)
    tasks = ""
    curdate = ""
    for ln in dat[0]['content'].splitlines():
        if '*' not in ln and len(ln.strip()) > 0:
            curdate = ln.strip() + ' ' + str(reference_date.year)
        elif '*' in ln:
            if ln.count(",") == 3:
                ln = ln + ",1"
            tasks = tasks + "\n" + curdate + "," + ln
    tasks = tasks.splitlines()
    tasks = [ln.replace("*", "") for ln in tasks if len(ln.split(",")) == 6]
    tasks = pd.read_csv(StringIO('\n'.join(tasks)), header=None,
                        names=['Date', 'EndTime', 'Category', 'Desc', 'Duration', 'Point'])
    tasks['Date'] = pd.to_datetime(tasks['Date'], format="%b %d %Y")
    tasks.Category = tasks.Category.apply(lambda x: x.strip().lower())  # remove whitestrips
    tasks['DateStr'] = tasks.Date.apply(lambda x: dt.datetime.strftime(x, format="%b-%d"))
    tasks['Duration'] = pd.to_numeric(tasks.Duration, errors="coerce")  # coerce to numeric if any blank durations
    return tasks

tasks_all=get_tasks(reference_date)